In [ ]:
#First, make sure the notebook is aware of the workshop data sets
!pip install ase
!git clone https://github.com/icomse/5th_workshop_MachineLearning.git
import os
os.chdir('5th_workshop_MachineLearning/data')

## Regression as machine learning

Let's start with the simplest models we can! Plain old fitting to a line; the simplest model there can be. 

In [ ]:
import numpy as np
import matplotlib.pyplot as plt 
import pandas as pd 
%matplotlib inline

In [ ]:
# the names in sklearn tend to be pretty long, so we will import the individual
# objects rather than the whole library 

from sklearn.linear_model import LinearRegression

### Let's read in some data

This is some artificial data we will be playing around with to illustrate some important concepts.

In [ ]:
dlin = pd.read_csv('linmod.csv')

What does this data look like? Inspect directly and plot.

In [ ]:
dlin.head()

In [ ]:
dlin.plot(x='inputs',y='outputs',kind='scatter')
plt.show()

Now, let's train a linear model with the data! First, create the model. It starts out as just an empty object, that we need to fill.

In [ ]:
linmodel = LinearRegression()

Now look up the documentation as to what to do to fit the model. When you fit it, what do you get back? NOTE: `LinearRegression` expects that that the input X is a 2D arrray, of size `[n_features, n_samples]`, because it is designed to handle the general case that you are doing multiple variable linear regression. So if you pass in a 1D array, it will error. Read the error message you get carefully for a quick fix.

In [ ]:
X = dlin['inputs'].values.reshape(-1, 1)
Y = dlin['outputs']
linmodel.fit(X,Y)

OK, you have a model. We next want to plot the prediction of the model and the points on the same plot?

In [ ]:
xvals = np.linspace(10,70,100)
ypred = linmodel.predict(xvals.reshape(-1,1))  # use this to apply the model
plt.plot(xvals,ypred,c='k')
plt.scatter(X,Y)
plt.xlabel('inputs')
plt.ylabel('outputs')
plt.show()

What methods does the `LinearRegression` have? What information can you get from it?  What are the parameters of a 1D linear regression?  What is a measure of how good the model is?

In [ ]:
linmodel?

In [ ]:
print(linmodel.intercept_)
print(linmodel.coef_)
print(linmodel.score(X,Y))  # in this case, the score gives the R^2.

Let's play around with (over)fitting!  There's a little bit of curvature - in the data; maybe we should try a polynomial fit?

Let's actually just use a sample of the data set to train; we want to go up to the number of data points, and polynomial regression is numerically unstable with large numbers.  There are tools doing this type of split.

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.3)

We make a polynomial fit using sklearn by taking powers of the input features.  `scikit-learn` has a function for this.

In [ ]:
from sklearn.preprocessing import PolynomialFeatures

To use it, you create a `PolynomialFeatures` object of the degree you are interested in, then take the powers of the data using the fit_transform method of `PolynomialFeatures`. See example (you will have to supply your own X, which is the same input as `LinearRegression.fit` takes.

In [ ]:
pf = PolynomialFeatures(degree=2)
pX_train = pf.fit_transform(X_train)

In [ ]:
print(X_train[:5])
print(pX_train[:5])

it's a degree $\times$ n_data array; with each column the input data to power $n$.

Now, lets see how the model fits the data.  Inspect the model parameters and the $R^2$, and plot the fit and the curve.

In [ ]:
linmodel.fit(pX_train,Y_train)
print(linmodel.intercept_)
print(linmodel.coef_)
print('R^2 (train) =', linmodel.score(pX_train,Y_train))

In [ ]:
xvals = np.linspace(10,70,100)
px = pf.fit_transform(xvals.reshape(-1,1))
ypredict = linmodel.predict(px)
plt.plot(xvals,ypredict,c='k')
plt.scatter(X_train,Y_train)
plt.xlabel('inputs')
plt.ylabel('outputs')
plt.show()

Now test the model on the test set.

In [ ]:
pX = pf.fit_transform(X_test)
plt.plot(xvals,ypredict,c='k')
plt.scatter(X_test,Y_test)
plt.xlabel('inputs')
plt.ylabel('outputs')
print('R^2 =', linmodel.score(pX,Y_test))

Remember, it's the TESTING MSE (or $R^2$) that we want to minimize, not the training MSE.  If the training MSE is low, but the testing is high, then the model is overfit.  

**HACKING TIME**: Now, go back and try a higher polynomial degree. Note that polynomial fitting is not very numerically stable, so if you fit to too many points, then you start to have problems.  What problems do you have?

Can you plot $R^2$ (i.e. linmodel.score) for the test and training set versus polynomial degree?  What do you see?

Generally the training set will get a bit better as the model gets more complex, whereas the test model gets worse as the trained model starts to overfit.

### Multilinear regression

That's well and good, but that is just one input.  Usually, we have LOTS of features and want to use all of them.

In [ ]:
cld=pd.read_csv('HCEPDB_100K_cleaned.csv') # change where it is
# (but first you might refresh yourself on what is contained
# we'll use a smaller sample to make it go faster, and to make the problem a bit harder.
cld = cld[cld['pce']!=0] # clean out the data where PCE = 0 (not obtained)
cld_sample = cld.sample(n=1000,random_state=12345)

The basic principle (and code!) is the same as ordinary least squares.

In [ ]:
features = ['mass', 'voc', 'jsc', 'e_homo_alpha', 'e_gap_alpha', 'e_lumo_alpha']
observable = 'pce'
X = cld_sample[features]
Y = cld_sample[observable]

In [ ]:
# now fit the model
linmodel = LinearRegression() 
linmodel.fit(X,Y)
linmodel.score(X,Y)

Now, there are more coefficients! One for each feature

In [ ]:
linmodel.coef_

One think to take into account, though. What if the data has different units, or is different types?  How will the coefficients change? 

Now make a test-train split. We'll reserve most of the data for testing (we want to make it harder to fit!)

In [ ]:
train, test = train_test_split(cld_sample, test_size=0.95, random_state=123)

We use data standardization or normalization to remove this effect.  Now, it's not always the best thing to do.  If you have two variables that are the same units, it could be that the distances DO matter.  

Minimum, it makes sure that the model does not depend on the units used.

In [ ]:
from sklearn.preprocessing import StandardScaler

In [ ]:
scaler = StandardScaler()
scaler.fit(cld_sample[features])
train_standardized = pd.DataFrame(data=scaler.transform(train[features]),columns=features)
test_standardized = pd.DataFrame(data=scaler.transform(test[features]),columns=features)
# stick back in the unscaled observables
test_standardized['pce']=test['pce'].values 
train_standardized['pce']=train['pce'].values

For lasso and ridge regression, $R^2$ doesn't quite make sense, because the extra terms messes with calculation of the total sum of squares.  Instead, let's just look at the means squared error. For simple linear regression (no matter how many variables.  Also, the coefficients of simple linear regression can take into account the change of units (it's the shrinkage term that causes problems), but we'll do everything with standardized data to make it simpler. 

In [ ]:
from sklearn.metrics import mean_squared_error

## Part 1: Just getting it set up with more rigor: Multiple linear regression

In [ ]:
# train linear model 
MLR=LinearRegression()
MLR.fit(train[features],train[[observable]])

# make predictions on test and train set 
trainpred=MLR.predict(train[features])
testpred=MLR.predict(test[features])

#make parity plot 
plt.figure(figsize=(7,7))
plt.scatter(test[observable],testpred,color='r', label='Test')
plt.scatter(train[observable],trainpred, label='Training')
plt.plot(lw=4,color='black')
plt.legend()
plt.xlabel('Actual Output')
plt.ylabel('Predicted Output')

#calculate the test and train error
print("Train error",mean_squared_error(train[observable],trainpred))
print("Test error",mean_squared_error(test[observable],testpred))
print(MLR.coef_)

## Part 2: Ridge Regression (same data as Part 1)

* The ridge coefficients minimize $RSS + \lambda \sum_{j=1}^{p}\beta_j^2$
* There is an additional **penalty** in error for having large coefficients!
* Note: ISLR shows the tuning parameter as $\lambda$, but it is $\alpha$ in `sk-learn`
* Goal here: train models as a function of the regularization parameter 
* The X's **must** be be standardized for ridge regression to work correctly 
* Some methods in `sk_learn` also do automatic selection of shrinkage coefficient - you can try those out afterwards. 
* **For the next section, I suggest on your own you test out what normalization in ridge does, by executing the following code first WITHOUT and then WITH normalization** 

In [ ]:
from sklearn.linear_model import Ridge, Lasso

### Example of a single instance of ridge regression


In [ ]:
ridge=Ridge()
a=2.0
ridge.set_params(alpha=a)
ridge.fit(train_standardized[features],train_standardized[observable])

In [ ]:
train_predict = ridge.predict(train_standardized[features])
test_predict = ridge.predict(test_standardized[features])
print(mean_squared_error(train_standardized[observable],train_predict))
print(mean_squared_error(test_standardized[observable],test_predict))
print(ridge.coef_)

In [ ]:
ridge=Ridge()
a=8.0
ridge.set_params(alpha=a)
ridge.fit(train_standardized[features],train_standardized[observable])
train_predict = ridge.predict(train_standardized[features])
test_predict = ridge.predict(test_standardized[features])
print(mean_squared_error(train_standardized[observable],train_predict))
print(mean_squared_error(test_standardized[observable],test_predict))
print(ridge.coef_)

### Hyperparameters

$\alpha$ (or $\lambda$) is an example of a *hyperparameter*.  It's a variable that will change exactly what the model is doing.  The parameters of the models are the coefficients of the ridge regression; each choice of parameter gives a different model, but each choice of hyperparameters leads to a different model. Deep learning methods often have *many* hyperparameters. Tuning hyperparameters is a key part of building machine learning models.  So let's tune a hyperparameter!

**Hacking**: 
1. Redo the linear regression with standardized data.  How do the coefficents change when you add regression?
2. Vary $\alpha$ and see how the predictions and coefficients change.

In [ ]:
plt.figure(figsize=(7,7))
plt.scatter(train_standardized[observable],train_predict, label='Training')
plt.scatter(test_standardized[observable],test_predict,color='r', label='Test')
plt.plot([-2.5,10],[-2.5,10],lw=4,color='black')
plt.legend()
plt.xlabel('Actual Output')
plt.ylabel('Predicted Output')
plt.show()

### Example of searching the $\alpha$ ($\lambda$ in the notes) space in Ridge Regression

In [ ]:
# RR vs lambda (based on sklearn tutorial)
coefs = []
trainerror = []
testerror = []

model = Ridge()

lambdas = np.logspace(-4,5,100)
# loop over lambda values (strength of regularization and find the errors)
for l in lambdas:
    model.set_params(alpha=l)
    model.fit(train_standardized[features],train_standardized[observable])
    coefs.append(model.coef_)
    trainerror.append(mean_squared_error(train_standardized[observable],model.predict(
        train_standardized[features])))
    testerror.append(mean_squared_error(test_standardized[observable],model.predict(
        test_standardized[features])))

In [ ]:
# what is being plotted here? 
plt.figure(figsize=(10,3))
plt.subplot(121)
plt.plot(lambdas,coefs)
plt.xscale('log')
plt.xlabel('$\lambda$')
plt.ylabel('coefs')
plt.title('RR coefs vs $\lambda$')
plt.subplot(122)
plt.plot(lambdas,trainerror,label='train error')
plt.plot(lambdas,testerror,label='test error')
plt.xscale('log')
plt.ylim([0.01,1.5])
plt.xlabel('$\lambda$')
plt.ylabel('error')
plt.legend(loc=1)
plt.title('error vs $\lambda$')
plt.show()

**Pause and discuss with a partner**: What is happening in these plots? 

The model is best (least overfit, best test MSE) at between $\lambda$ ($\alpha$) = $10^1$ and $10^2$, specifically:

In [ ]:
lambdas[np.argmin(testerror)] #the lambda where the MSE is minimized

## Part 3: LASSO regression  (same data as Part 1)

* The LASSO improves over ridge regression by also providing a variable selection tool!
* The LASSO minimizer is $RSS + \lambda \sum_{j=1}^{p}\lvert\beta_j\rvert$

**Hacking time**: Plot the LASSO regression coefficients versus the magnitude of the shrinkage term, in the same way the ridge regression was done above.

In [1]:
# also based on sklearn tutorials
# think about what is happening in this cell?
coefs = []
trainerror = []
testerror = []
lambdas = np.logspace(-5,3,100)
model = Lasso() # we already standardized the data

# loop over lambda values (strength of regularization)
for l in lambdas:
    model.set_params(alpha=l,max_iter=100000)
    model.fit(train_standardized[features],train_standardized[observable])
    coefs.append(model.coef_)
    trainerror.append(mean_squared_error(train_standardized[observable],model.predict(
        train_standardized[features])))
    testerror.append(mean_squared_error(test_standardized[observable],model.predict(
        test_standardized[features])))

plt.figure(figsize=(10,3))
plt.subplot(121)
plt.plot(lambdas,coefs)
plt.xscale('log')
plt.xlabel('$\lambda$')
plt.ylabel('coefs')
plt.title('Lasso coefs vs $\lambda$')

plt.subplot(122)
plt.plot(lambdas,trainerror,label='train error')
plt.plot(lambdas,testerror,label='test error')
plt.xscale('log')
plt.xlabel('$\lambda$')
plt.ylabel('error')
plt.xlim(1e-4,1e0)
plt.ylim(0.2,2.0)
plt.legend()
plt.title('error vs $\lambda$')
plt.show()

NameError: name 'np' is not defined

In [ ]:
min_lambda = lambdas[np.argmin(testerror)]  # at labmbda =0.28 the test error is minimized.
print(min_lambda)

In [ ]:
coefs[np.argmin(testerror)]

In [ ]:
coefs[np.argmin(testerror)]==0.0

Some of the coefficients are zero. Specifically:

In [ ]:
np.array(features)[coefs[np.argmin(testerror)]==0.0]  # need to convert to numpy array to use masking. 

So LASSO suggest that the best model would have just:

In [ ]:
np.array(features)[coefs[np.argmin(testerror)]!=0.0] 


### General linear models 

In the last class, we generated model based on the hypothesis that the $PCE$ of a candidate organic photovoltaic can be modeled as a contribution of the molecule's $mass$, $VOC$ and $E_{LUMO}$ values:  $PCE = \beta_0 + \beta_1\times mass + \beta_2\times VOC + \beta_3 \times E_{LUMO}$

The extension from multiple linear regression to general additive models is straightforward; it uses the same module `LinearRegression` module.

Let's first look at the residuals, versus the function, and then the residuals versus each of the variables, and see if there is any nonlinearity. Check the RSE of this model for future reference.

In [ ]:
train,test = train_test_split(cld, test_size=0.95)
features = ['mass', 'voc', 'e_lumo_alpha']
observable = 'pce'
train_X = train[features]
train_Y = train[['pce']]
test_X = test[features]
test_Y = test[['pce']]

In [ ]:
model = LinearRegression() 
model.fit(train_X,train_Y)

In [ ]:
predictions = model.predict(test_X)

In [ ]:
print("RSE =",mean_squared_error(test_Y,model.predict(test_X)))
plt.hexbin(test_Y,predictions-test_Y,cmap='hot')
plt.show()

What sort of nonlinear model do these plots suggest to try?  Try something, and compare $R^2$, RSE, and plot the residuals of the new model and the new model.

In [ ]:
# for complicated reasons, we have to add columns to the origina dataframe , not the cleaned dataframe 
# (basically, since the cleaned file is a slice of a dataframe, not a dataframe itself)
cld['voc2'] = cld['voc']**2  
cld['lumo2'] = cld['e_lumo_alpha']**2
cld['voc3'] = cld['voc']**3
cld['lumo3'] = cld['e_lumo_alpha']**3
cld = cld[cld['pce']!=0]  # clean out bad PCE's again. 

In [ ]:
train,test = train_test_split(cld, test_size=0.95)
features = ['mass','voc','voc2','voc3','e_lumo_alpha','lumo2','lumo3']
observable = 'pce'
train_X = train[features]
train_Y = train[['pce']]
test_X = test[features]
test_Y = test[['pce']]

model = LinearRegression() 
model.fit(train_X,train_Y)

print("RSE =",mean_squared_error(test_Y,model.predict(test_X)))
plt.hexbin(test_Y,model.predict(test_X)-test_Y,cmap='hot')
plt.show()

**Hacking time!**: What other general linear models, more complicated or simpler, can you test? How do you know if they are better?

## Classification using: Logistic regression

Let's use a database of perovskite stability data patients to demonstrate logistic regression.  Perovskites have the general formula $ABX_3$, where $A$ and $B$ are cations and $X$ is an anion.

The variables are described as follows:
- ABX3: chemical formula of the compound
- exp_label: is it stable?  -1 is no, 1 is yes
- is_train: is it training data? -1 is no, 1 is yes
- nA: $n_A$, oxiation state of A
- nB: $n_B$, oxidation state of B
- nX: $n_X$, oxidations state of X
- rA (Ang):	$r_A$, ionic radius of A in Angstroms
- rB (Ang):	$r_B$, ionic radius of B in Angstroms
- rX (Ang):	$r_X$, ionic radius of X in Angstroms
- t: $t$, Goldschmidt tolerance factor, $\frac{r_A + r_X}{\sqrt{2}\left(r_B+r_X\right)}$
- tau: $\tau$, Bartel et al. tolerance factor $\frac{r_X}{r_B}-n_A\left(n_A - \frac{r_A/r_B}{\ln(r_A/r_B)}\right)$
- t_pred: Whether $t$ predicts it will be a perovskite, -1 is no, 1 is yes
- tau_pred:	Whether $\tau$ predicts it will be a perovskite ($\tau < 4.18$), -1 is no, 1 is yes
- tau_prob:	Probability that the compund it is a perovskite, according to $\tau$	

nA,nB,nX,rA,rB,rX are the features used for training.

Let's start by just looking at the means and standard deviations to see what it looks like.

In [ ]:
perov=pd.read_csv('perovskite_data.csv')
perov.head()

In [ ]:
# examine the input features which are not categorical. 
training_features = ['nA','nB','nX','rA (Ang)','rB (Ang)', 'rX (Ang)']
table1=np.mean(perov[training_features],axis=0)
table2=np.std(perov[training_features],axis=0)
df = pd.DataFrame(columns=['means','stds'])
df['means'] = table1
df['stds'] = table2
df

Let's inspect the other data to make sure we know what is going on.

In [ ]:
perov['is_train'].hist()

`LogisticRegression` is in `sklearn.linear_model` and has many of the same inputs and outputs as `LinearRegression`

In [ ]:
from sklearn.linear_model import LogisticRegression

Train a `LogisticRegression` model on the six inputs, and score it. (What is the score here? is it $R^2$)?

In [ ]:
perov_training = perov[perov['is_train']==1]
perov_testing =  perov[perov['is_train']==-1]
inputData = perov_training[['nA','nB','nX','rA (Ang)','rB (Ang)', 'rX (Ang)']]
outputData = (perov_training['exp_label']+1)/2  # rescale to 0,1, though it ends up not mattering.

inputData_test = perov_testing[['nA','nB','nX','rA (Ang)','rB (Ang)', 'rX (Ang)']]
outputData_test = (perov_testing['exp_label']+1)/2  # rescale to 0,1, though it ends up not mattering.

In [ ]:
logit1=LogisticRegression(solver='liblinear')
#we specify the liblinear solver to avoid a warning about not specifying
logit1.fit(inputData,outputData)
logit1.score(inputData,outputData)

`Score` for logistic regression is just the fraction that are right.

In [ ]:
np.mean(logit1.predict(inputData)==outputData)

Now, check on the test data.

In [ ]:
logit1.score(inputData_test,outputData_test)

If we were to use NO inputs, just the outputs, we would predict that the percent that are stable would be the same as the percent in our population, or about 50/50

In [ ]:
np.mean(outputData)

So we are getting noticably better than random!

Let's look at the documentation for the function.

In [ ]:
LogisticRegression?

#### Model performance
What are the percent who are correctly classified by this model?  What are the true positive, true negative, false positive, and false negative?  Try using the `confusion_matrix` function of `sklearn.metrics`. How well does the model predict which women don't have diabetes?  How well does it predict which women do?

In [ ]:
from sklearn.metrics import confusion_matrix
cm = confusion_matrix(logit1.predict(inputData),outputData)
print(cm)

In [ ]:
confusion_matrix?

The format is: 
[[ True Negative  False Negative],
 [ False Positive True Positive]]

Look at coefficients.  Can you tell which ones are most important in predicting stability?  Why or why not? 

In [ ]:
logit1.coef_

**Hacking Time:** How does a prediction with standardized variables do versus one without standardized ones? What do the coeficients look like before and after? 

Is all of the data linearly independent?

What are those other columns doing there?  What is `tau_pred`?

In [ ]:
cm = confusion_matrix((perov_training['tau_pred']+1)/2,outputData)
print(cm)
cm = confusion_matrix((perov_testing['tau_pred']+1)/2,outputData_test)
print(cm)

In [ ]:
np.mean((perov_training['tau_pred']+1)/2==outputData)

In [ ]:
np.mean((perov_testing['tau_pred']+1)/2==outputData_test)